# Energy and economic output

This notebook aims to show the link (if not causal relationship) between energy consumption (or production) and GDP output.

This notebook is coded in Python and relies on a few libraries (or packages), which we load below

In [1]:
import pandas as pd                      # This library is for data wrangling
import seaborn as sns                    # This is one library for plotting
import matplotlib.pyplot as plt          # Yet another library for visualization
import plotly.express as px              # This is another library for plots 
import warnings                          # This is to remove warning to keep output clean
warnings.filterwarnings('ignore')

## Data
### Data sources

We need two types of data and hence resort to two sources:
- for **energy** production and consumption, we use data compiled by **BP**. While it is undeniably a corporate source, it is widely considered as reliable, especially for non renewable energies. For renewables, the post https://www.resilience.org/stories/2017-08-02/how-bp-is-minimizing-renewable-energy-in-its-statistical-review/ contends that the company purposefully minimizes the numbers. For our study, this will not be a major issue.
- for **economic data**, several sources are possible, including the **International Monetary Fund** and the **World Bank**, and we will work with the latter. Both provide country-level data up to 1980 and 1960, respectively.

### Energy data
**NOTE**: the dataset is quite large (250K+ rows), so the importation takes a few seconds.

In [2]:
url = 'https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/xlsx/energy-economics/statistical-review/bp-stats-review-2021-consolidated-dataset-narrow-format.xlsx'
data_energy_sources = pd.read_excel(url) # Reads the online file 
data_energy_sources                      # Shows the result

,Country,Year,ISO3166_alpha3,ISO3166_numeric,Region,SubRegion,OPEC,EU,OECD,CIS,Var,Value
0,Algeria,1965,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_twh,0.000000
1,Algeria,1966,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_twh,0.000000
2,Algeria,1967,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_twh,0.000000
3,Algeria,1968,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_twh,0.000000
4,Algeria,1969,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_twh,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
252911,Zimbabwe,2016,ZWE,716.0,Africa,Eastern Africa,0.0,0.0,0.0,0.0,pop,14.030338
252912,Zimbabwe,2017,ZWE,716.0,Africa,Eastern Africa,0.0,0.0,0.0,0.0,pop,14.236599
252913,Zimbabwe,2018,ZWE,716.0,Africa,Eastern Africa,0.0,0.0,0.0,0.0,pop,14.438812
252914,Zimbabwe,2019,ZWE,716.0,Africa,Eastern Africa,0.0,0.0,0.0,0.0,pop,14.645473


One interesting column is the **Var**, which specifies which variable is produced. Let's have a look at the list.

In [3]:
data_energy_sources['Var'].unique().tolist() # List of possible fields

['wind_twh',
 'wind_mtoe',
 'wind_ej',
 'solar_twh',
 'solar_mtoe',
 'solar_ej',
 'renewables_mtoe',
 'renewables_ej',
 'ren_power_twh',
 'ren_power_mtoe',
 'ren_power_ej',
 'refthru_kbd',
 'refcap_kbd',
 'rareearths_kt',
 'primary_mtoe',
 'primary_gj_pc',
 'primary_ej',
 'oilreserves_bbl',
 'oilprod_ngl_kbd',
 'oilprod_mt',
 'oilprod_kbd',
 'oilprod_crudecond_kbd',
 'oilcons_mtoe',
 'oilcons_mt',
 'oilcons_kbd',
 'oilcons_ej',
 'nuclear_twh',
 'nuclear_mtoe',
 'nuclear_ej',
 'lithium_kt',
 'liqcons_kbd',
 'hydro_twh',
 'hydro_mtoe',
 'hydro_ej',
 'graphite_kt',
 'gasreserves_tcm',
 'gasprod_mtoe',
 'gasprod_ej',
 'gasprod_bcm',
 'gasprod_bcfd',
 'gascons_mtoe',
 'gascons_ej',
 'gascons_bcm',
 'gascons_bcfd',
 'ethanol_prod_pj',
 'ethanol_prod_ktoe',
 'ethanol_prod_kboed',
 'ethanol_prod_kbd',
 'ethanol_cons_pj',
 'ethanol_cons_ktoe',
 'ethanol_cons_kboed',
 'ethanol_cons_kbd',
 'electbyfuel_oil',
 'electbyfuel_gas',
 'electbyfuel_coal',
 'elect_twh',
 'cobalt_kt',
 'coalprod_mtoe',
 '

This is a looong list. We see that the suffixes tell the units in which the values are expressed. For instance, "ej" refers to **exajoule**, which is 10^18 joules while "pt" to **petajoule**, which is 10^15 joules. The joule is a classical unit used to quantify amounts of energy.

Since most categories are expressed in joules, we will keep these variables and discard the rest.

In [8]:
data_energy_sources = data_energy_sources[data_energy_sources['Var'].str.endswith('j')]
data_energy_sources

,Country,Year,ISO3166_alpha3,ISO3166_numeric,Region,SubRegion,OPEC,EU,OECD,CIS,Var,Value
10006,Algeria,1965,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_ej,0.000000
10007,Algeria,1966,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_ej,0.000000
10008,Algeria,1967,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_ej,0.000000
10009,Algeria,1968,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_ej,0.000000
10010,Algeria,1969,DZA,12.0,Africa,Northern Africa,1.0,0.0,0.0,0.0,wind_ej,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
245378,US,2016,USA,840.0,North America,North America,0.0,0.0,1.0,0.0,biodiesel_cons_pj,259.853847
245379,US,2017,USA,840.0,North America,North America,0.0,0.0,1.0,0.0,biodiesel_cons_pj,247.374569
245380,US,2018,USA,840.0,North America,North America,0.0,0.0,1.0,0.0,biodiesel_cons_pj,237.209821
245381,US,2019,USA,840.0,North America,North America,0.0,0.0,1.0,0.0,biodiesel_cons_pj,225.887978
